In [1]:
+import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.utils import np_utils

%matplotlib inline

import check_miss_value
m0o

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [5]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [6]:
train.Sex.values.reshape(-1, 1).shape

(891, 1)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [10]:
LE1 = LabelEncoder()

In [11]:
train.Sex = LE1.fit_transform(train.Sex.values.reshape(-1, 1))

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
train['Sex'].value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [13]:
train.Sex.value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [14]:
train.Ticket.value_counts()

1601             7
CA. 2343         7
347082           7
CA 2144          6
347088           6
3101295          6
382652           5
S.O.C. 14879     5
2666             4
347077           4
17421            4
PC 17757         4
113760           4
4133             4
349909           4
W./C. 6608       4
19950            4
LINE             4
113781           4
110152           3
110413           3
SC/Paris 2123    3
35273            3
347742           3
24160            3
345773           3
363291           3
PC 17760         3
C.A. 31921       3
F.C.C. 13529     3
                ..
371060           1
7598             1
240929           1
2694             1
36967            1
349208           1
349206           1
113028           1
350417           1
374746           1
2628             1
C.A. 29178       1
347076           1
363592           1
349219           1
330980           1
111427           1
349223           1
330932           1
345770           1
113773           1
349251      

In [15]:
# Ticket は後で削除する

In [16]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [17]:
LE2 = LabelEncoder()

In [18]:
from sklearn.preprocessing import Imputer

In [19]:
impubter = Imputer(missing_values=np.nan, strategy='most_frequent')

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [20]:
# 文字列型は Imputerで入れられないので、最頻値を直接入れる
train.Embarked = train.Embarked.fillna('S')

In [21]:
train.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [22]:
train.Embarked = LE2.fit_transform(train.Embarked.values.reshape(-1, 1))

In [23]:
train.Embarked.value_counts()

2    646
0    168
1     77
Name: Embarked, dtype: int64

In [24]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.036847,-0.057527,-0.001652,0.012658,0.013128
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.077221,-0.035322,0.081629,0.257307,-0.167675
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.369226,0.083081,0.018443,-0.549500,0.162098
Sex,0.042939,-0.543351,0.131900,1.000000,0.093254,-0.114631,-0.245489,-0.182333,0.108262
Age,0.036847,-0.077221,-0.369226,0.093254,1.000000,-0.308247,-0.189119,0.096067,-0.030394
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.308247,1.000000,0.414838,0.159651,0.068230
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.189119,0.414838,1.000000,0.216225,0.039798
Fare,0.012658,0.257307,-0.549500,-0.182333,0.096067,0.159651,0.216225,1.000000,-0.224719
Embarked,0.013128,-0.167675,0.162098,0.108262,-0.030394,0.068230,0.039798,-0.224719,1.000000


In [25]:
train.corr()[train.corr() > 0.2]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Survived,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.257307,NaN
Pclass,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Sex,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Age,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
SibSp,NaN,NaN,NaN,NaN,NaN,1.000000,0.414838,NaN,NaN
Parch,NaN,NaN,NaN,NaN,NaN,0.414838,1.000000,0.216225,NaN
Fare,NaN,0.257307,NaN,NaN,NaN,NaN,0.216225,1.000000,NaN
Embarked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [26]:
train.corr()[train.corr() < -0.2]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Survived,NaN,NaN,-0.338481,-0.543351,NaN,NaN,NaN,NaN,NaN
Pclass,NaN,-0.338481,NaN,NaN,-0.369226,NaN,NaN,-0.549500,NaN
Sex,NaN,-0.543351,NaN,NaN,NaN,NaN,-0.245489,NaN,NaN
Age,NaN,NaN,-0.369226,NaN,NaN,-0.308247,NaN,NaN,NaN
SibSp,NaN,NaN,NaN,NaN,-0.308247,NaN,NaN,NaN,NaN
Parch,NaN,NaN,NaN,-0.245489,NaN,NaN,NaN,NaN,NaN
Fare,NaN,NaN,-0.549500,NaN,NaN,NaN,NaN,NaN,-0.224719
Embarked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.224719,NaN


In [27]:
# PassengerId, Name, Ticket, Cabin　を削除
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [28]:
check_miss_value.check_miss_value(train)

C:\Users\owner\codexa\900_Kaggle\001_Titanic\check_miss_value.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  miss_table.coulmns = ['欠損数', '%']


,欠損数,%
Survived,0,0.00000
Pclass,0,0.00000
Sex,0,0.00000
Age,177,19.86532
SibSp,0,0.00000
Parch,0,0.00000
Fare,0,0.00000
Embarked,0,0.00000


In [29]:
train.Age = train.Age.fillna(train.Age.median())

In [30]:
check_miss_value.check_miss_value(train)

,欠損数,%
Survived,0,0.0
Pclass,0,0.0
Sex,0,0.0
Age,0,0.0
SibSp,0,0.0
Parch,0,0.0
Fare,0,0.0
Embarked,0,0.0


In [31]:
# testデータ
check_miss_value.check_miss_value(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,86,20.574163
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,1,0.239234
Cabin,327,78.229665


In [32]:
test.Age = test.fillna(test.Age.median())

In [33]:
test.Fare = test.fillna(test.Fare.median())

In [34]:
check_miss_value.check_miss_value(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000
Cabin,327,78.229665


In [35]:
# Name, Ticket, Cabin　を削除
test = test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [38]:
test.Sex = LE1.fit_transform(test.Sex.values.reshape(-1, 1))

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [39]:
test.Embarked = LE2.fit_transform(test.Sex.values.reshape(-1, 1))

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [40]:
test.Sex.value_counts()

1    266
0    152
Name: Sex, dtype: int64

In [42]:
test.Embarked.value_counts()

1    266
0    152
Name: Embarked, dtype: int64

In [43]:
test.corr()

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked
PassengerId,1.000000,-0.026751,0.023245,0.003818,0.043080,0.023245
Pclass,-0.026751,1.000000,0.108615,0.001087,0.018721,0.108615
Sex,0.023245,0.108615,1.000000,-0.099943,-0.159120,1.000000
SibSp,0.003818,0.001087,-0.099943,1.000000,0.306895,-0.099943
Parch,0.043080,0.018721,-0.159120,0.306895,1.000000,-0.159120
Embarked,0.023245,0.108615,1.000000,-0.099943,-0.159120,1.000000


In [45]:
# picle ファイルに出力
train.to_pickle('pd_train.pk2')
test.to_pickle('pd_test.pk2')